# Torsion angle scan with rdkit & xtb [Spinning Coral]

## Include project directory into Python path

In [ ]:
import sys
import os

# Get the absolute path of the project root (where the src folder is)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
src_path = os.path.join(project_root, "src")

# Add the src folder to sys.path
if src_path not in sys.path:
    sys.path.append(src_path)

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms
import copy
import pandas as pd
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields
from rdkit.Chem import rdMolTransforms
from pathlib import Path

import subprocess
import re

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.ipython_3d=True

In [ ]:
%load_ext autoreload
%autoreload 2
from confgen import example_function
from confgen.io.reader.sdf import sdf_to_mol_list
from confgen.widgets.mol_visulization import draw_overlapped_mols

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Generate Molecule From SMILES

In [ ]:
from jsme_notebook import JSMENotebook
# Use JSME To Draw Molecule and Copy SMILES
jsme = JSMENotebook()

In [ ]:
smiles="CCCC"
mol = Chem.MolFromSmiles(smiles)
mol = AllChem.AddHs(mol)
print(f"Coordinates:{"3D" if (mol.GetNumConformers() > 0 and mol.GetConformer().Is3D() == True) else "2D"}, Number of Conformers: {mol.GetNumConformers()}")
mol

In [ ]:
AllChem.EmbedMolecule(mol)
print(f"Coordinates:{"3D" if (mol.GetNumConformers() > 0 and mol.GetConformer().Is3D() == True) else "2D"}, Number of Conformers: {mol.GetNumConformers()}")
print(f"Initial angle: {rdMolTransforms.GetDihedralDeg(mol.GetConformer(),0,1,2,3)}")
mol

In [ ]:
# Create a copy of original mol
m2=copy.deepcopy(mol)
# Get Force Field Parameters for ligand
mp2 = AllChem.MMFFGetMoleculeProperties(m2)

# Setup Dihedral Scan Range
energy=[]
confid=0
angles=range(-180,180,5)

In [ ]:
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp2)
    # Specify atom to screen
    ff2.MMFFAddTorsionConstraint(0,1,2,3, False, angle - .2, angle + .2, 1000.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())

    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)


df_rdkit = pd.DataFrame({'angle':angles, 'mmff_energy':energy})
print(f"Coordinates:{"3D" if (mol.GetNumConformers() > 0 and mol.GetConformer().Is3D() == True) else "2D"}, Number of Conformers: {mol.GetNumConformers()}")

In [ ]:
dfrdkit.plot( x="angle", y="mmff_energy")

# xTB Torsion Scan

In [ ]:
smiles="CCCC"
mol = Chem.MolFromSmiles(smiles)
mol = AllChem.AddHs(mol)
AllChem.EmbedMolecule(mol)
mol

In [ ]:
angles=range(-180,180,5)

In [ ]:
mol_name="butane"
w_dir = Path.cwd().joinpath("torsion_scan")
w_dir.mkdir(parents=True, exist_ok=True)
w_dir

In [ ]:
## Prepare xtb geom
rdkit_atoms_index="0,1,2,3"
xtb_atoms_index='1,2,3,4' #set atoms to define the dihedral - NB: xtb indexes start at 1, rdkit at 0
for idx,deg in enumerate(angles):
    print(idx, deg)
    
    # Write constraint file
    const_mark = f"p{abs(deg)}" if deg <= 0 else f"n{abs(deg)}"
    with open(w_dir.joinpath(f"dih_const_{const_mark}.inp"),"w") as fh:
        fh.write(f"$constrain\nforce constant=1.0\ndihedral: {xtb_atoms_index},{float(deg)}\n$end\n")
    
    # Now write the xtb input file:
    if idx == 0:
        input_file_path = w_dir.joinpath(f"{mol_name}_initial_geom.sdf")
        output_file_path = w_dir.joinpath(f"{mol_name}_{const_mark}.out")
        # ToDo: generate mol can be contrain to specific dihedral and pre-optimized using FF
        with Chem.SDWriter(str(input_file_path)) as sdf_w:
            sdf_w.write(mol)
    else:
        input_file_path = output_file_path.with_name(f"{output_file_path.stem}.xtbopt.sdf")
        if input_file_path.is_file():
            output_file_path = w_dir.joinpath(f"{mol_name}_{const_mark}.out")
        else:
            print(f"Failed to find {input_file_path}.")
            break

    # Run xTB Opt.
    try:
        command = f"cd {w_dir}; xtb {input_file_path} --opt tight --namespace {output_file_path.stem} --input dih_const_{const_mark}.inp > {output_file_path}"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        # Print the result
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
        
    # if idx==3:
    #     break

In [ ]:
mol_dict = {}
energies = []
for idx,deg in enumerate(angles):
    # Write constraint file
    const_mark = f"p{abs(deg)}" if deg <= 0 else f"n{abs(deg)}"
    output_sdf_path = w_dir.joinpath(f"{mol_name}_{const_mark}.xtbopt.sdf")
    # print(const_mark, output_sdf_path)
    xtb_opt_mol = sdf_to_mol_list(output_sdf_path)[0]

    # Extract energy from title
    title = xtb_opt_mol.GetProp("_Name")
    match = re.search(r'energy:\s*(-?\d+\.\d+)', title)
    energy_value = match.group(1)  # Extract the matched number
    # print(f"{deg} Extracted value: {energy_value}")
    energies.append([int(deg), float(energy_value)])
    # break

print(energies)

In [ ]:
df_xtb = pd.DataFrame(energies, columns = ["angle", "xtb_energy"])
df_xtb.plot( x="angle", y="xtb_energy")

In [ ]:
sns.lineplot(x="angle",y="xtb_energy",data=df_xtb, color="g")
ax2 = plt.twinx()
sns.lineplot(x="angle",y="mmff_energy",data=dfrdkit, color="b", ax=ax2)

# Resources

- [Crystallography Open Database](https://www.crystallography.net/cod/index.php)

# Inspiration

- [Spinning coral](https://pschmidtke.github.io/blog/posts/post-with-code/2021-01-25-cod-and-torsion-angles/2021-01-25-cod-and-torsion-angles.html)